# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcac0a18850>
├── 'a' --> tensor([[ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430]])
└── 'x' --> <FastTreeValue 0x7fcac0a18490>
    └── 'c' --> tensor([[-0.8729, -0.7575,  1.5053,  0.2012],
                        [ 0.6696,  1.5430, -0.7437, -0.5160],
                        [-0.8160, -0.3840, -0.0750,  1.0428]])

In [4]:
t.a

tensor([[ 0.0834,  0.4345,  0.3227],
        [-1.7807, -0.9803, -2.5430]])

In [5]:
%timeit t.a

59.1 ns ± 0.00924 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcac0a18850>
├── 'a' --> tensor([[1.4466, 0.6280, 0.0753],
│                   [1.3692, 0.5717, 0.4973]])
└── 'x' --> <FastTreeValue 0x7fcac0a18490>
    └── 'c' --> tensor([[-0.8729, -0.7575,  1.5053,  0.2012],
                        [ 0.6696,  1.5430, -0.7437, -0.5160],
                        [-0.8160, -0.3840, -0.0750,  1.0428]])

In [7]:
%timeit t.a = new_value

59.2 ns ± 0.0491 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430]]),
    x: Batch(
           c: tensor([[-0.8729, -0.7575,  1.5053,  0.2012],
                      [ 0.6696,  1.5430, -0.7437, -0.5160],
                      [-0.8160, -0.3840, -0.0750,  1.0428]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0834,  0.4345,  0.3227],
        [-1.7807, -0.9803, -2.5430]])

In [11]:
%timeit b.a

51.2 ns ± 0.0614 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5890,  0.9104,  0.0503],
               [ 1.1886, -0.0129, -0.9002]]),
    x: Batch(
           c: tensor([[-0.8729, -0.7575,  1.5053,  0.2012],
                      [ 0.6696,  1.5430, -0.7437, -0.5160],
                      [-0.8160, -0.3840, -0.0750,  1.0428]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.152 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

861 ns ± 0.295 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 24.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 267 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fca0bcb71f0>
├── 'a' --> tensor([[[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]],
│           
│                   [[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]],
│           
│                   [[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]],
│           
│                   [[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]],
│           
│                   [[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]],
│           
│                   [[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]],
│           
│                   [[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]],
│           
│                   [[ 0.0834,  0.4345,  0.3227],
│                    [-1.7807, -0.9803, -2.5430]]])
└── 'x' --> <FastTreeValue 0x7fca0bcb7280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fca0bc9ec70>
├── 'a' --> tensor([[ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430],
│                   [ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430],
│                   [ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430],
│                   [ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430],
│                   [ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430],
│                   [ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430],
│                   [ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430],
│                   [ 0.0834,  0.4345,  0.3227],
│                   [-1.7807, -0.9803, -2.5430]])
└── 'x' --> <FastTreeValue 0x7fca0bc9edc0>
    └── 'c' --> tensor([[-0.8729, -0.7575,  1.5053,  0.2012],
                        [ 0.6696,  1.5430, -0.7437, -0.5160],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.3 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]],
       
               [[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]],
       
               [[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]],
       
               [[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]],
       
               [[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]],
       
               [[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]],
       
               [[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]],
       
               [[ 0.0834,  0.4345,  0.3227],
                [-1.7807, -0.9803, -2.5430]]]),
    x: Batch(
           c: tensor([[[-0.8729, -0.7575,  1.5053,  0.2012],
                       [ 0.6696,  1.5430, -0.7437, -0.5160],
                       [-0.8160, -0.3840, -0.0750,  1.0428]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 270 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430],
               [ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430],
               [ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430],
               [ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430],
               [ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430],
               [ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430],
               [ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430],
               [ 0.0834,  0.4345,  0.3227],
               [-1.7807, -0.9803, -2.5430]]),
    x: Batch(
           c: tensor([[-0.8729, -0.7575,  1.5053,  0.2012],
                      [ 0.6696,  1.5430, -0.7437, -0.5160],
                      [-0.8160, -0.3840, -0.0750,  1.0428],
                      [-0.8729, -0.7575,  1.5053,  0.2012],
                      [ 0.6696,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 271 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 1.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
